# CKD Plan Designer Tutorial

## Install CKDPlanDesigner Package

In [34]:
# install package from source code
! pip install -e project_code/

Obtaining file:///Users/nicholasdeveau/projects/bmi210/bmi210Project/project_code
  Attempting uninstall: CKDPlanDesigner
    Found existing installation: CKDPlanDesigner 0.0.0
    Uninstalling CKDPlanDesigner-0.0.0:
      Successfully uninstalled CKDPlanDesigner-0.0.0
  Running setup.py develop for CKDPlanDesigner


## Load Data
- This tutorial uses the freely available synthea example dataset
    - Download it here: https://synthetichealth.github.io/synthea-sample-data/downloads/synthea_sample_data_csv_apr2020.zip
    - unzip it to SYNTHEA_PATH
 

In [3]:
import pandas as pd
import os

from CKDPlanDesigner.utils import data_preprocessing as pre

SYNTHEA_PATH = '/Users/nicholasdeveau/projects/bmi210/bmi210Project_data/synthea_data/'

#### Here we build our patient dataframe
- Feature extraction and preprocessing occurs in utils/data_preprocessing.py
- This contains a vecor for each patient containing the required information for plan designing 

In [9]:
patients_df = pre.preprocess_synthea_data(SYNTHEA_PATH)

/Users/nicholasdeveau/projects/bmi210/bmi210Project/project_code/src/CKDPlanDesigner/utils/data_preprocessing.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sel_cond_df['VALUE'] = 1


In [10]:
patients_df.head()

,patient_id,t2d,depression,bmi,hypertension,smoking,dob,race,ethnicity,gender,zip,eGFR,age
0,c8403116-d069-4130-aa02-39954fc971b5,True,False,False,False,False,1967-03-26,white,nonhispanic,F,NaN,66.9,55
1,73f124a4-e69e-4b8c-a447-e72d23dc54be,True,False,False,True,True,1967-06-14,white,nonhispanic,F,2170.0,70.5,55
2,31459cc4-45ed-4dee-b160-9f1a148a7724,False,False,True,False,False,1968-08-15,white,nonhispanic,M,NaN,75.4,54
3,799d0738-d5cd-4161-bdb8-e3c49e21ec66,False,False,True,True,True,1939-12-12,white,nonhispanic,M,2360.0,82.0,83
4,bae73dce-20fa-4a6a-8e46-b2ed541f9296,True,True,False,False,True,1958-04-27,white,nonhispanic,M,NaN,83.6,64


## Working With a Patient

In [11]:
from CKDPlanDesigner.models import patients, plans, interventions
%load_ext autoreload
%autoreload patients
%autoreload plans
%autoreload interventions

In [13]:
# convert one row of the patient_df to a dictionary. This 
# dictionary will be our config
patient1_config = patients_df.to_dict(orient='index')[5]

In [15]:
# select a  patient and convert feature vector to a dictionary
# we pass this dictionary to the patient class on instantiation
patient1_config = patients_df.to_dict(orient='index')[13]
p1 = patients.Patient(**patient1_config)

# now select another patient
patient2_config = patients_df.to_dict(orient='index')[44]
# patient2_config = patients_df.to_dict(orient='index')[15]
p2 = patients.Patient(**patient2_config)

#### Look at patient 1's characteristics

In [17]:
p1.patient_config

{'patient_id': '844d1c39-6ca2-48df-a3de-f9a12633f69c',
 't2d': True,
 'depression': False,
 'bmi': False,
 'hypertension': False,
 'smoking': False,
 'dob': '1963-07-20',
 'race': 'white',
 'ethnicity': 'nonhispanic',
 'gender': 'F',
 'zip': nan,
 'eGFR': 8.9,
 'age': 59}

In [18]:
# we automatically assign a CKD stage when a patient is created
p1.stage

### Generate a care plan

In [20]:
p1.generate_careplan()
print(p1.careplan.plan_name)

Plan: Smart Dialysis


In [35]:
# Here we see the components of a care plan personalized for this patient.
p1.careplan.list_components(show_savings=False)

Plan Components


['Dialysis', 'Education and Engagement', 'Nephro Engagement']

In [36]:
# Custom education content specific to the smart dialysis plan.
# This content can be modified by a non-technical user simply by
# editing the text in configs/ix_config.yaml
p1.careplan.all_components[1].desc_long

'Discuss desicion wrt. palliaive care vs. dialysis, emphasizing there is little evidence for outcomes benefit with dialysis at this stage. Also discuss patient on benefits with home dialysis and the simplicity of it. Emphasize that it is not hard at all.'

### Try it out for a different patient
- This patient is old and has a bad prognosis (e.g. stage 5 / end-stage CKD). Check out the difference in care plan!
- Because of this, we should expect a plan that focuses on palliative care.

In [30]:
# note high age and low eGFR
p2.patient_config

{'patient_id': 'c9942738-49f3-4459-818f-8da20072ff76',
 't2d': True,
 'depression': True,
 'bmi': False,
 'hypertension': True,
 'smoking': True,
 'dob': '1935-10-27',
 'race': 'white',
 'ethnicity': 'nonhispanic',
 'gender': 'M',
 'zip': 2474.0,
 'eGFR': 9.5,
 'age': 87}

In [31]:
# stage 5
p2.stage

In [29]:
p2.generate_careplan()
p2.careplan.list_components(show_savings=False)

Plan Components


['ComfortAndPain']